In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from umap import UMAP


# <font color="grey"> Graph Representation Learning at Scale</font>

#### <font color="grey">  Table of Contents</font>

1. <a href='#chapter1'>Structure</a>
2. <a href='#chapter2'>Datasets</a>
3. <a href='#chapter3'>Graphs</a>
4. <a href='#chapter4'>Patches</a>
5. <a href='#chapter5'>Embedding</a>
6. <a href='#chapter6'>Alignment</a>
7. <a href='#chapter7'>Hierarchical alignment</a>
8. <a href='#chapter8'>Visualisation</a>

###  <a id='chapter1'> <font color="grey">1. Structure </font></a>

There are five main parts to the package, organised as follows.

```
l2gv2/
├── datasets/
├── graphs/
├── patch/
├── embedding/
└── align/
    ├── l2g/
    └── geo/
```

A brief overview of the contents:

* ```datasets``` contains interfaces are provided for various common benchmark datasets. 
* ```graphs``` contains wrappers for graphs represented as lists of edges in pytorch-geometric ```data.edge_index``` format. These implemented features such as fast adjacency look-up and a variety of algorithms on graphs. Eventually, one could replace these with more sophisticad formats such as [Raphtory](https://www.raphtory.com/) graphs. 
* ```patch``` directory contains datastructures to represent patches and patch graphs, as well as methods to subdivide a graph into patches. 
* ```embedding``` contains various graph embedding methods, including Graph Autoencoders (GAE) and [Variational Graph Autoencoders (VGAE)](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.models.VGAE.html).
* ```align``` contains two methods to compute the alignment of patches into a single graph embedding: eigenvalue synchronisation based on the [Local2Global](https://link.springer.com/article/10.1007/s10994-022-06285-7) algorithm, and the new method based on learning the alignment using a one-layer neural network.

###  <a id='chapter2'> <font color="grey">2. Datasets </font></a>

There are currently three datasets available:

* ```Cora```: The [Cora dataset](https://graphsandnetworks.com/the-cora-dataset/) is a dataset of 2708 scientific publications divided into 7 classes. The citation network consists of 5429 directed edges. To each publication / node there is an associated 1433-dimensional feature vector indicating the presence or absence of certain words. This dataset is accessed through the pytorch-geometric [Planetoid](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.datasets.Planetoid.html) dataset.
* ```as-733```: The [SNAP autonomous systems AS-733](https://snap.stanford.edu/data/as-733.html) dataset contains 733 daily snapshots that span an interval of 785 days from November 8 1997 to January 2 2000. In each of these datasets, nodes represent autonomous systems and edges indicate whether communication has taken place. The resulting graph is undirected.
* ```mag240```: The [MAT240M](https://ogb.stanford.edu/docs/lsc/mag240m/) dataset is a large heterogeneous academic citation graph.
* ```elliptic```: The [Elliptic dataset](https://www.kaggle.com/datasets/ellipticco/elliptic-data-set) maps Bitcoin transactions to real entities belonging to licit categories (exchanges, wallet providers, miners, licit services, etc.) versus illicit ones (scams, malware, terrorist organizations, ransomware, Ponzi schemes, etc.). The task on the dataset is to classify the illicit and licit nodes in the graph. The graph consists of 203,769 nodes representing transactions and 234,355 directed edges representing payments flows.
A case study is the paper [Anti-Money Laundering in Bitcoin: Experimenting with Graph
Convolutional Networks for Financial Forensics](https://arxiv.org/pdf/1908.02591) by Weber et.al.
* ```Dgraph```: [DGraph](https://dgraph.xinye.com/dataset) is a real world financial graph assembled for anomaly detection. This graph is described in the paper [DGraph: A Large-Scale Financial DAtaset for Graph Anomaly Detection](https://arxiv.org/abs/2207.03579).

These are accessed using the ```get_dataset``` function. The format of the datasets follows closely the [pytorch-geometric convention](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.data.Dataset.html). If the data is not locally available, it is retreived and preprocessed into a specified directory. In the case of temporal (dynamic) graphs, the graphs are returned as an iterable over time slices. Ultimately, the graphs can be exported into formats such as raphtory or used to initialize ```TGraph``` instances (see <a href='#chapter3'>Chapter 3: Graphs</a>). Currently, only ```Cora```, ```as-733``` (temporal) and ```DGraph``` are available. The ```DGraph``` dataset needs to be downloaded manually and a path to the zip file provided as argument when first initialising it.

In [9]:
from l2gv2.datasets import get_dataset

In [24]:
cora = get_dataset("Cora")
print(cora[0])

Loading edge and node data from memory


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [11]:
# First transform data into raphtory format, then networkx for plotting.
G = cora.to("raphtory").to_networkx()

In [12]:
as733 = get_dataset("as-733")

2025-03-11 09:48:47,154 - INFO - Downloading dataset tarball...
2025-03-11 09:49:02,499 - INFO - Download complete.
2025-03-11 09:49:02,500 - INFO - Extracting dataset tarball...
2025-03-11 09:49:03,023 - INFO - Extraction complete.
Processing...
2025-03-11 09:49:03,023 - INFO - Processing raw text files into Polars DataFrames...
2025-03-11 09:49:04,038 - INFO - Processing edges complete. Parquet files saved to /Users/u1774790/Projects/G2007/code/L2Gv2/data/as733/processed.
Done!


Loading edge and node data from memory
Loading edge and node data from memory


In [13]:
g = as733.to("raphtory")

print("Stats on the graph structure:")

number_of_nodes = g.count_nodes()
number_of_edges = g.count_edges()
total_interactions = g.count_temporal_edges()

print("Number of nodes (AS nodes):", number_of_nodes)
print("Number of unique edges (src,dst):", number_of_edges)
print("Total interactions (edge updates):", total_interactions)

print("Stats on the graphs time range:")

earliest_datetime = g.earliest_date_time
latest_datetime = g.latest_date_time

print("Earliest datetime:", earliest_datetime)
print("Latest datetime:", latest_datetime)

Stats on the graph structure:
Number of nodes (AS nodes): 7716
Number of unique edges (src,dst): 45645
Total interactions (edge updates): 11965533
Stats on the graphs time range:
Earliest datetime: 1997-11-08 00:00:00+00:00
Latest datetime: 2000-01-02 00:00:00+00:00


In [33]:
from torch_geometric.datasets import DGraphFin
from pathlib import Path
root = "../data/dgraph/"
data = DGraphFin(root)
data[0]


Data(x=[3700550, 17], edge_index=[2, 4300999], y=[3700550], edge_type=[4300999], edge_time=[4300999], train_mask=[3700550], val_mask=[3700550], test_mask=[3700550])

In [31]:
data.raw_paths

['../data/dgraph/raw/DGraphFin.zip']

In [37]:
# The Dgraph dataset requires registration. It needs to be downloaded from the website and placed in the data folder.
dgraph = get_dataset("DGraph", root="../data/dgraph", source_file="../data/DGraphFin.zip")
print(dgraph[0])

Loading edge and node data from memory


Data(x=[3700550, 17], edge_index=[2, 4300999], y=[3700550], edge_type=[4300999], edge_time=[4300999], train_mask=[3700550], val_mask=[3700550], test_mask=[3700550])


###  <a id='chapter3'> <font color="grey">3. Graphs </font></a>

There are three wrappers for graphs that were taken over from the local2global package: ```TGraph```, ```NPGraph``` and ```JitGraph```. These include, among other things, methods for fast adjacency look-up and various optimizations.

In [8]:
from l2gv2.graphs import TGraph

In [ ]:
tg = TGraph(cora[0].edge_index, edge_attr=cora[0].edge_attr, x=cora[0].x)
print(tg.adj_index)
print(tg.x)

In a future iteration one can think about consolidating this part by having graphs represented in some existing graph package like Raphtory.

###  <a id='chapter4'> <font color="grey">4. Patches </font></a>

A patch can equivalently refer to a subgraph or to an embedding of this subgraph. As a set of points, a patch is represented using the ```Patch``` class. A ```Patch``` object has the properties ```nodes```, ```index``` and ```coordinates```. ```nodes``` is simply a list of the nodes from the original graph that are present in the patch. ```index``` is a dict that maps each node to an index into ```coordinates```, which is just a list of coordinates. For example, if a graph embedding consists of four nodes in two dimensions as follows, and a patch is represented by the solid circles, then the corresponding object would have the following properties:

![Patch](./images/square_patch.png)


In [10]:
from l2gv2.patch.patches import Patch

In [ ]:
p = Patch([0,2,3], np.array([[0., 0.], [1., 0.], [1., 1.]]))
print(p.coordinates)
print(p.nodes)
print(p.index)

In [12]:
from l2gv2.example import generate_patches, random_transform_patches, plot_patches

In [ ]:
patches = generate_patches(n_points = 200, n_clusters=10)
n_patches = len(patches)
print(f"First 5 coordinates: {patches[0].coordinates[:5]}")
print(f"First 5 nodes: {patches[0].nodes[:5]}")
print(f"Indices: {patches[0].index}")

The crucial feature of patches is that different patches may reference the same node in an underlying graph, but with different coordiantes. A typical 

In [ ]:
# Apply random shifts and rotations to the patches
transformed_patches = random_transform_patches(patches)
plot_patches(patches, transformed_patches)

The patches from the nodes of a **patch graph**, where two nodes are connected by an edge if the patches contain overlapping nodes. The alignment tasks consists of making the correponding coordinates overlap as much as possible.

###  <a id='chapter5'> <font color="grey">5. Embedding </font></a>

The coordinates of a patch come from an embedding of a graph into Euclidean space. 
For the embedding, we use the architecture of a Variational Graph Autoencoder. Given a graph $G=(V,E)$ with $|V|=n$ nodes and node features ${x}_i\in \mathbb{R}^d$, $i\in [n]$, denote by ${X}=[{x}_1,\dots,{x}_n]^T\in \mathbb{R}^{n\times d}$ the features matrix and by $A=(a_{ij})\in \{0,1\}^{n\times n}$ the adjacency matrix of the graph. The **encoder** produces latent representations ${z}_i\in \mathbb{R}^k$ for $i\in [n]$, which are sampled from the inference model
\begin{equation*}
  q({z}_i \ | \ {X},{A}) = \mathcal{N}({z}_i \ | \ {\mu}_i,\mathrm{diag}({\sigma}_i)).
\end{equation*}
The means $\mu_i$ and variances $\mathrm{diag}({\sigma}_i)$ are parametrized using an encoder network, for example, a graph convolutional neural network (GCN). Denoting by ${Z}=[{z}_1,\dots,{z}_n]^T$ the matrix of latent represenations and by ${\mu}$ and ${\sigma}$ the matrices representing the means and variances, we have
\begin{equation*}
  {\mu} = \mathrm{GCN}_{\mu}({X},{A}), \quad \quad \log {\sigma} = \mathrm{GCN}_{\sigma}({X},{A}).
\end{equation*}
The **generative model** is a distribution on the adjacency matrix,
\begin{equation*}
  p({A}\ | \ {Z}) = \prod_{i,j} p(a_{ij} \ | \ {z}_i,{z}_j).
\end{equation*}
It is convenient to use
\begin{equation*}
  p(a_{ij}=1 \ | \ {z}_i,{z}_j) = \sigma({z}_i^T{z}_j),
\end{equation*}
where $\sigma$ is the logistic sigmoid. In order to train the model, we optimize the evidence lower bound
\begin{equation*}
  \mathcal{L} = \mathbb{E}_{q({Z}\ | \ {X},{A})}[\log p({A}\ | \ {Z})]-\mathrm{D}_{\mathrm{KL}}(q({Z}\ | \ {X},{A}) \ \| \ p({Z})).
\end{equation*}

In [15]:
from l2gv2.embedding.gae import VGAE
from l2gv2.embedding.train import train
from l2gv2.embedding.gae.utils.loss import VGAE_loss

In [ ]:
cora[0].y

In [ ]:
model = VGAE(dim=64, hidden_dim=128, num_features=tg.x.shape[1]).to(device)
model = train(cora[0], model, loss_fun=VGAE_loss, num_epochs=200, verbose=False, lr=0.001)
with torch.no_grad():
    model.eval()
    coordinates = model.encode(cora[0]).to("cpu").numpy()

In [ ]:
reducer = UMAP(n_components=2, random_state=42)
umap_data = reducer.fit_transform(coordinates)

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(umap_data[:, 0], umap_data[:, 1], c=cora[0].y, cmap='viridis', alpha=0.5)
plt.title('UMAP Visualization Cora data')
plt.show()

###  <a id='chapter6'> <font color="grey">6. Alignment </font></a>

We have a way of representing patches and a way of embedding them. The next step is to compute the alignment based on the patch graph. There are two methods for this, we focus on the 'new' one.

In [23]:
from l2gv2.align import get_aligner

In [ ]:
geo_aligner = get_aligner("geo", patches=transformed_patches)
embedding = geo_aligner.get_aligned_embedding()

In [ ]:
plot_patches(geo_aligner.patches, patches)

In [ ]:
plt.scatter(embedding[:, 0], embedding[:, 1])
plt.show()

###  <a id='chapter7'> <font color="grey">7. Hierarchical alignment </font></a>

To be done.

###  <a id='chapter8'> <font color="grey">8. Visualisation </font></a>

For the visualisation, it is convenient to use external packages such as Heimdall.